In [3]:
import numpy as np
import pandas as pd
import gensim
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
df_filter = pd.DataFrame()

def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)

        vocab_quotes = chunk.quotation.apply(gensim.utils.simple_preprocess)
        tagged_data_2 = [TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(vocab_quotes)]
        
        # On définit le modèle, il y a beaucoup de paramètres à étudier pour optimiser
        #model = gensim.models.Doc2Vec(vector_size=300, window=10, min_count=5, workers=4, epochs=20)
        model = gensim.models.Doc2Vec(vector_size=100, dbow_words= 1, dm=0,  window=5, seed=1337, min_count=5, workers=4,
                              alpha=0.025, min_alpha=0.025)
        
        # On construit le vocab nécessaire au training 
        model.build_vocab(tagged_data_2)
        
        # On entraine le modèle
        model.train(tagged_data_2, total_examples=model.corpus_count, epochs=20)
        
        # On cherche les 10 documents les plus similaires à celui-ci. 
        tokens = "Climate change is a big problem.".split()
        new_vector = model.infer_vector(tokens)
        sims = model.dv.most_similar([new_vector])
        print(sims)
        
        print(type(sims))
        sims = np.array(sims)
        sims = sims[:,0]
        sims = sims.astype(int)
        
        print(type(sims))
        print(sims)
        
        df_filter = chunk.iloc[sims]
        

with pd.read_json('quotes-2020.json.bz2', lines=True, compression='bz2', chunksize=100000) as df_reader:
    for chunk in df_reader:
        process_chunk(chunk)
        
print(df_filter)

Processing chunk with 100000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
[(11190, 0.7680513858795166), (91448, 0.7215105891227722), (8558, 0.7070029377937317), (86148, 0.702319324016571), (92592, 0.6939815282821655), (40799, 0.6926105618476868), (10317, 0.6893508434295654), (17129, 0.6839941740036011), (76454, 0.681468665599823), (29958, 0.6807796359062195)]
<class 'list'>
<class 'numpy.ndarray'>
[11190 91448  8558 86148 92592 40799 10317 17129 76454 29958]
Processing chunk with 100000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')
[(37942, 0.8083097338676453), (96891, 0.7581177949905396), (97327, 0.7416934370994568), (24263, 0.7303981184959412), (86447, 0.7205579876899719), (4347, 0.7029721140861511), (20135, 0.7010608911514282), (4323, 0.6997730135917664), (81043, 0.6848158240318298), (37964, 0.6819020